# Upper Confidence Bound BO


In [1]:
# Import the class
import torch
import yaml

## Customizing the UpperConfidenceBound Generator
First lets examine the possible options that we can specify for the
UpperConfidenceBound generator. We can use these keys to customize optimization.

In [2]:
from xopt.generators import get_generator_and_defaults
ucb_gen, ucb_options = get_generator_and_defaults("upper_confidence_bound")
print(yaml.dump(ucb_options.dict()))

acq:
  beta: 2.0
  monte_carlo_samples: 128
  proximal_lengthscales: null
  use_transformed_proximal_weights: true
model:
  function: !!python/name:xopt.generators.bayesian.models.standard.create_standard_model ''
  kwargs:
    use_conservative_prior_lengthscale: false
    use_conservative_prior_mean: false
    use_low_noise_prior: false
n_initial: 3
optim:
  max_travel_distances: null
  num_restarts: 5
  raw_samples: 20
  sequential: true
  use_nearby_initial_points: true



In [3]:
from xopt.resources.test_functions.rosenbrock import make_rosenbrock_vocs, evaluate_rosenbrock
from xopt import Xopt, Evaluator
import pandas as pd

vocs = make_rosenbrock_vocs(2)

generator_options = ucb_gen.default_options()
generator_options.optim.num_restarts = 20
generator_options.optim.max_travel_distances = [0.1, 0.1]
generator_options.acq.beta = 2.0
generator_options.model.kwargs.use_low_noise_prior = True
#generator_options.acq.proximal_lengthscales = [0.01, 0.01]

evaluator = Evaluator(function=evaluate_rosenbrock)
generator = ucb_gen(vocs, generator_options)


In [4]:
X = Xopt(generator=generator, evaluator=evaluator, vocs=vocs)

for i in range(100):
    X.step()
    print(f"step {i}: best: {X.data['y'].min()}")


step 0: best: 13.474850101576134
step 1: best: 13.474850101576134
step 2: best: 13.474850101576134
step 3: best: 13.474850101576134
step 4: best: 8.677283806473813
step 5: best: 8.677283806473813
step 6: best: 8.677283806473813
step 7: best: 7.876998446094431
step 8: best: 7.876998446094431
step 9: best: 7.876998446094431
step 10: best: 4.334983793622543
step 11: best: 4.334983793622543
step 12: best: 4.334983793622543
step 13: best: 4.334983793622543
step 14: best: 4.334983793622543
step 15: best: 2.1975277264098656
step 16: best: 0.2729029035451416
step 17: best: 0.2729029035451416
step 18: best: 0.2729029035451416
step 19: best: 0.04020730431276022
step 20: best: 0.04020730431276022
step 21: best: 0.04020730431276022
step 22: best: 0.04020730431276022
step 23: best: 0.04020730431276022
step 24: best: 0.04020730431276022
step 25: best: 0.04020730431276022
step 26: best: 0.04020730431276022
step 27: best: 0.04020730431276022
step 28: best: 0.04020730431276022
step 29: best: 0.04020730

## View output data

In [5]:
X.data

x0        x1           y  xopt_runtime  xopt_error
1   -0.650258  0.094941   13.474850      0.000014       False
2   -0.665808  1.326833   80.837944      0.000006       False
3   -1.308704 -1.001066  741.785887      0.000004       False
4   -0.908704 -0.601066  207.221511      0.000013       False
5   -0.508704 -1.001066  160.997361      0.000012       False
..        ...       ...         ...           ...         ...
98   1.111341  1.243932    0.020234      0.000013       False
99   0.942652  0.871097    0.033898      0.000014       False
100  1.024702  1.044814    0.003315      0.000013       False
101  1.200948  1.444814    0.041024      0.000013       False
102  1.264524  1.593966    0.072530      0.000013       False

[102 rows x 5 columns]

In [6]:
# plot results
ax = X.data.plot(*vocs.variable_names)
ax.set_aspect("equal")

## Visualize model used by upper confidence bound
Models are kept in a list, in this case that list has one element, the model
created for the objective `y1`.


In [7]:
from matplotlib import pyplot as plt  # plot model predictions

data = X.data

bounds = generator.vocs.bounds
model = generator.train_model(generator.data)

# create mesh
n = 200
x = torch.linspace(*bounds.T[0], n)
y = torch.linspace(*bounds.T[1], n)
xx, yy = torch.meshgrid(x, y)
pts = torch.hstack([ele.reshape(-1, 1) for ele in (xx, yy)]).double()

outputs = generator.vocs.output_names
with torch.no_grad():
    post = model.posterior(pts)

    mean = post.mean
    std = torch.sqrt(post.variance)

    fig, ax = plt.subplots()
    ax.plot(*data[vocs.variable_names].to_numpy().T, "+C1")
    c = ax.pcolor(xx, yy, mean.reshape(n, n))
    fig.colorbar(c)
    ax.set_title(f"Posterior mean: {outputs[0]}")

    fig2, ax2 = plt.subplots()
    ax2.plot(*data[vocs.variable_names].to_numpy().T, "+C1")
    c = ax2.pcolor(xx, yy, std.reshape(n, n))
    fig2.colorbar(c)
    ax2.set_title(f"Posterior std: {outputs[0]}")

/usr/share/miniconda/envs/xopt-dev/lib/python3.9/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /home/conda/feedstock_root/build_artifacts/pytorch-recipe_1660136119739/work/aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [8]:
ax = X.data.plot(y="y", logy=True)

In [9]:
# Cleanup
!rm dump.yaml

rm: cannot remove 'dump.yaml': No such file or directory
